In [1]:
# Challenge 3 Prediction Bot/Human Social Media Accounts

#Importing required libraries
import time
import pandas as pd
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
#Reading data into Pandas Dataframe
data = pd.read_csv(r'D:\ML_Cyber\Dataset_Challenge3.csv')

In [3]:
#Understanding data
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37438 entries, 0 to 37437
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   default_profile         37438 non-null  bool   
 1   default_profile_image   37438 non-null  bool   
 2   favorites_count         37438 non-null  int64  
 3   followers_count         37438 non-null  int64  
 4   friends_count           37438 non-null  int64  
 5   geo_enabled             37438 non-null  bool   
 6   id                      37438 non-null  float64
 7   lang                    29481 non-null  object 
 8   location                37435 non-null  object 
 9   statuses_count          37438 non-null  int64  
 10  verified                37438 non-null  bool   
 11  average_tweets_per_day  37438 non-null  float64
 12  account_age_days        37438 non-null  int64  
 13  account_type            37438 non-null  object 
dtypes: bool(4), float64(2), int64(5), obje

,favorites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days
count,37438.000000,3.743800e+04,3.743800e+04,3.743800e+04,3.743800e+04,37438.000000,37438.000000
mean,12302.062183,3.703098e+05,4.445925e+03,1.221485e+17,2.107339e+04,6.891668,3022.128052
std,33923.650237,2.470829e+06,4.954520e+04,3.004180e+17,6.732977e+04,24.580116,1013.695870
min,0.000000,0.000000e+00,0.000000e+00,4.180000e+02,0.000000e+00,0.000000,483.000000
25%,362.000000,3.500000e+01,3.700000e+01,9.957306e+07,1.336250e+03,0.495000,2316.250000
50%,2066.000000,3.650000e+02,2.960000e+02,3.991474e+08,4.209000e+03,1.488000,3219.000000
75%,8879.000000,8.440250e+03,8.930000e+02,2.453826e+09,1.715375e+04,5.528000,3888.000000
max,885123.000000,1.216415e+08,4.343060e+06,1.120000e+18,2.771910e+06,1269.055000,5158.000000


In [4]:
#Finding the correlation of features:
data.corr()

,default_profile,default_profile_image,favorites_count,followers_count,friends_count,geo_enabled,id,statuses_count,verified,average_tweets_per_day,account_age_days
default_profile,1.000000,0.115994,-0.088897,-0.089969,-0.048898,-0.273882,0.309865,-0.131859,-0.264979,-0.082269,-0.459317
default_profile_image,0.115994,1.000000,-0.035908,-0.018062,-0.010599,-0.095831,0.014109,-0.027591,-0.058531,-0.020461,-0.047478
favorites_count,-0.088897,-0.035908,1.000000,-0.024431,0.066202,0.135524,-0.049551,0.264001,0.004998,0.275568,0.024098
followers_count,-0.089969,-0.018062,-0.024431,1.000000,0.048482,0.042550,-0.059866,0.031302,0.273493,0.010845,0.119114
friends_count,-0.048898,-0.010599,0.066202,0.048482,1.000000,0.028229,-0.029059,0.077872,0.051031,0.059842,0.043834
geo_enabled,-0.273882,-0.095831,0.135524,0.042550,0.028229,1.000000,-0.248482,0.090194,0.196153,0.057770,0.280077
id,0.309865,0.014109,-0.049551,-0.059866,-0.029059,-0.248482,1.000000,-0.085926,-0.190823,-0.020600,-0.730932
statuses_count,-0.131859,-0.027591,0.264001,0.031302,0.077872,0.090194,-0.085926,1.000000,0.079066,0.929172,0.115308
verified,-0.264979,-0.058531,0.004998,0.273493,0.051031,0.196153,-0.190823,0.079066,1.000000,0.027558,0.350272
average_tweets_per_day,-0.082269,-0.020461,0.275568,0.010845,0.059842,0.057770,-0.020600,0.929172,0.027558,1.000000,0.009868


In [5]:
#Getting the details of Language and Location features:
print('Location Description: \n', data['location'].describe(), '\n')
print('Language Description: \n', data['lang'].describe())

Location Description: 
 count       37435
unique      12047
top       unknown
freq        13023
Name: location, dtype: object 

Language Description: 
 count     29481
unique       48
top          en
freq      21450
Name: lang, dtype: object


In [6]:
# Dropping 'Location' Since it has very high 'unknown' values in addition to very high number of unique values, with low correlation with other features.
data = data.drop(columns ='location')

#Dropping 'Lang' as it has highly biased data with low correlation with other features. And is not logically a valuable feature to determine Human vs Bots.
data = data.drop(columns = 'lang')

In [7]:
# Converting categorical features to numerical values with LabelEncoder from scikit-learn.
le = preprocessing.LabelEncoder()
dataset = data.apply(le.fit_transform)

In [8]:
#Transformed Dataset
print(dataset.info())
print(dataset.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37438 entries, 0 to 37437
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   default_profile         37438 non-null  int64
 1   default_profile_image   37438 non-null  int64
 2   favorites_count         37438 non-null  int64
 3   followers_count         37438 non-null  int64
 4   friends_count           37438 non-null  int64
 5   geo_enabled             37438 non-null  int64
 6   id                      37438 non-null  int64
 7   statuses_count          37438 non-null  int64
 8   verified                37438 non-null  int64
 9   average_tweets_per_day  37438 non-null  int64
 10  account_age_days        37438 non-null  int64
 11  account_type            37438 non-null  int32
dtypes: int32(1), int64(11)
memory usage: 3.3 MB
None
       default_profile  default_profile_image  favorites_count  \
count     37438.000000           37438.000000     37438.0

In [9]:
#Splitting the dataset into Features and Target Variable and applying scaler function from scikit-learn:
X = dataset.drop(columns = 'account_type')
y = dataset['account_type']
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X.values)
y = dataset['account_type']


In [10]:
#Splitting dataset into Training and Testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y, shuffle=True)

In [11]:
#Performing kNN model  5 fold cross validation:
start_time = time.time()
k_acc_scores = []
k_cv_scores = []
k_values = [i for i in range(1,20,2)] #evaluating values of k as all odd numbers between 1 and 20

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train,y_train)
    pred = knn.predict(X_test)
    k_acc_scores.append("k({}) = {}".format(k, accuracy_score(y_test, pred)))
    cv_scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    k_cv_scores.append(cv_scores.mean())

end_time = time.time()
duration = end_time-start_time
print("Time Elapsed: {} ".format(duration))

Time Elapsed: 59.743908643722534 


In [12]:
#Evaluating Performance metrics for kNN Classifier model:
print(max(k_acc_scores))
print(max(k_cv_scores))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

k(9) = 0.8410790598290598
0.8412020033388983
[[1855  630]
 [ 579 4424]]
              precision    recall  f1-score   support

           0       0.76      0.75      0.75      2485
           1       0.88      0.88      0.88      5003

    accuracy                           0.84      7488
   macro avg       0.82      0.82      0.82      7488
weighted avg       0.84      0.84      0.84      7488



In [15]:
#Perfoming SVM model:
start_time = time.time()
parameters = {'gamma':[1,0.1]}
clf = GridSearchCV(svm.SVC(),parameters,cv=5, verbose=100,n_jobs=-1)
clf.fit(X_train,y_train)
end_time = time.time()
duration = end_time - start_time
print("Time Elapsed: {} ".format(duration))

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:  4.3min remaining: 17.3min
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  4.4min remaining: 10.2min
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:  4.4min remaining:  6.5min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  4.4min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  4.4min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  4.5min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  4.5min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  6.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  6.6min finished
Time Elapsed: 581.2853815555573 


In [16]:
#Evaluating Performance Metrics for SVM model:
pred_svm = clf.predict(X_test)
print(confusion_matrix(y_test, pred_svm))
print(classification_report(y_test, pred_svm))

[[  91 2394]
 [   0 5003]]
              precision    recall  f1-score   support

           0       1.00      0.04      0.07      2485
           1       0.68      1.00      0.81      5003

    accuracy                           0.68      7488
   macro avg       0.84      0.52      0.44      7488
weighted avg       0.78      0.68      0.56      7488



In [17]:
#The kNN model has higher accuracy, precision and recalls compared to those of SVM Model.
#Hence, considering these and their computation times, kNN model proves to be better performing for the provided dataset.
